In [1]:
import ast, copy, dis
import nbimport

In [2]:
import Episode11

# Hazards of substitution

In [5]:
inline_lambda = Episode11.inline_lambda
transformer = inline_lambda('f', 'lambda x: (z := x ** 2, z + x + 1)[-1]')
test0 = transformer('''
def wonkification():
    z = 42
    f(99)
    return z
''')
print(ast.dump(test0, indent=2))

Name(id='x', ctx=Load())
-->
Constant(value=99)
Name(id='x', ctx=Load())
-->
Constant(value=99)
Call(
  func=Name(id='f', ctx=Load()),
  args=[
    Constant(value=99)],
  keywords=[])
-->
Subscript(
  value=Tuple(
    elts=[
      NamedExpr(
        target=Name(id='z', ctx=Store()),
        value=BinOp(
          left=Constant(value=99),
          op=Pow(),
          right=Constant(value=2))),
      BinOp(
        left=BinOp(
          left=Name(id='z', ctx=Load()),
          op=Add(),
          right=Constant(value=99)),
        op=Add(),
        right=Constant(value=1))],
    ctx=Load()),
  slice=UnaryOp(
    op=USub(),
    operand=Constant(value=1)),
  ctx=Load())
===
Module(
  body=[
    FunctionDef(
      name='wonkification',
      args=arguments(
        posonlyargs=[],
        args=[],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
      body=[
        Assign(
          targets=[
            Name(id='z', ctx=Store())],
          value=Constant(value=42)),


In [6]:
print(ast.unparse(test0))

def wonkification():
    z = 42
    ((z := (99 ** 2)), z + 99 + 1)[-1]
    return z


In [8]:
exec(compile(test0, '<string>', 'exec'))

In [9]:
wonkification()

9801

In [11]:
transformer2 = inline_lambda('f', 'lambda x: (print("howdee"), x ** 2 + x + 1)[-1]')

In [12]:
%%mycell transformer2
def sillify():
    return f(42)


Name(id='x', ctx=Load())
-->
Constant(value=42)
Name(id='x', ctx=Load())
-->
Constant(value=42)
Call(
  func=Name(id='f', ctx=Load()),
  args=[
    Constant(value=42)],
  keywords=[])
-->
Subscript(
  value=Tuple(
    elts=[
      Call(
        func=Name(id='print', ctx=Load()),
        args=[
          Constant(value='howdee')],
        keywords=[]),
      BinOp(
        left=BinOp(
          left=BinOp(
            left=Constant(value=42),
            op=Pow(),
            right=Constant(value=2)),
          op=Add(),
          right=Constant(value=42)),
        op=Add(),
        right=Constant(value=1))],
    ctx=Load()),
  slice=UnaryOp(
    op=USub(),
    operand=Constant(value=1)),
  ctx=Load())
===


In [13]:
exec(compile(_12, '<string>', 'exec'))

In [14]:
sillify()

howdee


1807

In [15]:
%%mycell transformer2
def print(*args, **kws):
    pass

def stupify():
    return f(43)

stupify()

Name(id='x', ctx=Load())
-->
Constant(value=43)
Name(id='x', ctx=Load())
-->
Constant(value=43)
Call(
  func=Name(id='f', ctx=Load()),
  args=[
    Constant(value=43)],
  keywords=[])
-->
Subscript(
  value=Tuple(
    elts=[
      Call(
        func=Name(id='print', ctx=Load()),
        args=[
          Constant(value='howdee')],
        keywords=[]),
      BinOp(
        left=BinOp(
          left=BinOp(
            left=Constant(value=43),
            op=Pow(),
            right=Constant(value=2)),
          op=Add(),
          right=Constant(value=43)),
        op=Add(),
        right=Constant(value=1))],
    ctx=Load()),
  slice=UnaryOp(
    op=USub(),
    operand=Constant(value=1)),
  ctx=Load())
===


In [16]:
exec(compile(_15, '<string>', 'exec'))

In [17]:
# Let's see if overloading print() has screwed output up...
'Nope.'

'Nope.'

In [19]:
print(ast.unparse(_15))

In [20]:
stupify()

1893

In [21]:
print = __builtins__.print
print(ast.unparse(_15))

def print(*args, **kws):
    pass

def stupify():
    return (print('howdee'), 43 ** 2 + 43 + 1)[-1]
stupify()


# Putting lambda substitution into a module

In [32]:
import sublambda
from importlib import reload
reload(sublambda)

<module 'sublambda' from '/Users/jon/git/ohop/ohop/sublambda.py'>

In [33]:
%%mycell sublambda.build_sublambdas
f lambda x: x ** 2 + x + 1
g lambda y: y ** 3 - y

<function sublambda.inline_lambdas.<locals>.inliner(source)>

In [ ]:
# TODO: Test it out...